In [59]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

In [60]:
poet_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a obscure but extraordinary poet. You only write in korean. You write a poem about programming language. You usually use simile and metaphor of nature. ",
        ),
        ("human", "Write a poem about {programming_language}"),
    ]
)
poet_chain = poet_prompt | chat

In [61]:
poem_reviewer_propmpt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a famous poem reviewer. First, you read a poem gracefully. And you create your own characteristic reviews about the poem. You only create review in korean. You have a good insight for catching a implicit meanings of a poem.",
        ),
        ("human", "{poem}"),
    ]
)

poem_reviewer_chain = poem_reviewer_propmpt | chat

final_chain = {"poem": poet_chain} | poem_reviewer_chain

final_chain.invoke({"programming_language": "python"})

프로그래밍 언어 C++는
자연 속의 강한 바람과 같아

그 속에는 무한한 가능성이 펼쳐져
마치 끝없이 펼쳐진 하늘과 같아

변수와 함수가 마치 나무와 꽃처럼
하나의 생태계를 이루며

코드의 흐름은 강물처럼
끊임없이 흘러가는 시간과 같아

C++은 자연의 법칙을 닮아
우리에게 새로운 세계를 열어준다.이 시는 C++ 프로그래밍 언어를 자연의 원리와 비유하여 아름답게 표현한 작품이다. 시인은 C++의 무한한 가능성을 강한 바람과 펼쳐진 하늘에 비유하여 표현함으로써, 이 기술이 우리에게 제공하는 새로운 세계를 상상케 한다. 변수와 함수를 나무와 꽃으로 묘사하며, 코드의 흐름을 강물과 시간에 빗대어 표현함으로써, C++의 복잡한 구조와 흐름을 자연의 아름다움으로 이해할 수 있게 한다. 이 시는 기술과 자연이 어우러진 아름다운 표현으로, 독자에게 깊은 여운을 남기는 작품이라 할 수 있다.

AIMessageChunk(content='이 시는 C++ 프로그래밍 언어를 자연의 원리와 비유하여 아름답게 표현한 작품이다. 시인은 C++의 무한한 가능성을 강한 바람과 펼쳐진 하늘에 비유하여 표현함으로써, 이 기술이 우리에게 제공하는 새로운 세계를 상상케 한다. 변수와 함수를 나무와 꽃으로 묘사하며, 코드의 흐름을 강물과 시간에 빗대어 표현함으로써, C++의 복잡한 구조와 흐름을 자연의 아름다움으로 이해할 수 있게 한다. 이 시는 기술과 자연이 어우러진 아름다운 표현으로, 독자에게 깊은 여운을 남기는 작품이라 할 수 있다.')